## **Name:**  Goli Srikar
## **RegNo:** 22BCE9946
## **Slot No:** L43-L44

## Brill tagger

In [ ]:
import nltk
from nltk.tag import brill,brill_trainer,UnigramTagger
from nltk.tokenize import word_tokenize
from nltk.corpus import brown

In [ ]:
nltk.download('brown')
nltk.download('punkt_tab')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
train_data=brown.tagged_sents()[:3000]
test_data=brown.tagged_sents()[3000:]

In [ ]:
uni=UnigramTagger(train_data)
templates=brill.fntbl37()
brill=brill_trainer.BrillTaggerTrainer(uni,templates)
brill_tagger=brill.train(train_data)

sentence="The dog sat on the mat"
tokens=word_tokenize(sentence)
tagg=brill_tagger.tag(tokens)
print(tagg)
accuracy=brill_tagger.evaluate(test_data)
print(accuracy)

[('The', 'AT'), ('dog', 'NN'), ('sat', 'VBD'), ('on', 'IN'), ('the', 'AT'), ('mat', 'NN')]


<ipython-input-35-35d59e7a9465>:10: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  accuracy=brill_tagger.evaluate(test_data)


0.7876620759214776


## NER tagger

In [ ]:
import spacy

In [ ]:
model=spacy.load("en_core_web_sm")
text="Elon Musk, the CEO of SpaceX. The company launched in 2002. A machine to mars will cost $10000000 which is announced in paris. The United Nations also shown interest. "
doc=model(text)
for ent in doc.ents:
    print(ent.text,ent.label_)

Elon Musk PERSON
2002 DATE
10000000 MONEY
paris GPE
The United Nations ORG


## Tagger that tags Date and Money expressions

In [ ]:
import spacy
model=spacy.load("en_core_web_sm")
text="Elon Musk, the CEO of SpaceX. The company launched in 2002. A machine to mars will cost $10000000 which is announced in paris. The United Nations also shown interest. "
doc=model(text)
for ent in doc.ents:
  if(ent.label_=="MONEY" or ent.label_=="DATE"):
    print(ent.text,ent.label_)


/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


2002 DATE
10000000 MONEY


## HMM Tagger

In [ ]:
import nltk
from nltk.tag import hmm
from nltk.corpus import treebank

In [ ]:
nltk.download('treebank')
nltk.download('universal_tagset')

train_data=treebank.tagged_sents(tagset='universal')
test_data=treebank.tagged_sents(tagset='universal')[3000:]

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [ ]:
trainer=hmm.HiddenMarkovModelTrainer()
model=trainer.train(train_data)

sentence="He is a good hearted person".split()
tags=model.tag(sentence)
print(tags)

[('He', 'PRON'), ('is', 'VERB'), ('a', 'DET'), ('good', 'ADJ'), ('hearted', 'NOUN'), ('person', 'NOUN')]


In [ ]:
model.evaluate(test_data)

<ipython-input-21-8e756550dad9>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  model.evaluate(test_data)


0.9791927476796892

In [ ]:
pip install sklearn_crfsuite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.2 MB/s eta 0:00:00


### CRF tagger Conditional Random Fields

In [ ]:
import sklearn_crfsuite

# Extract features for each word
def word_features(sentence, index):
    word = sentence[index][0]
    features = {
        'word': word,
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_title': word.istitle(),
        'is_upper': word.isupper(),
        'is_digit': word.isdigit(),
        'prev_word': '' if index == 0 else     				sentence[index - 1][0],
        'next_word': '' if index == len(sentence) - 1 			else sentence[index + 1][0],
        'prefix-1': word[:1],
        'prefix-2': word[:2],
        'suffix-1': word[-1:],
        'suffix-2': word[-2:],
    }
    return features


In [ ]:
# Convert dataset to feature format
def sentence_features(sentence):
    return [word_features(sentence, i) for i in range(len(sentence))]
def sentence_labels(sentence):
    return [label for _, label in sentence]


In [ ]:
# Sample training dataset
train_data = [
    [('The','DET'), ('cat','NOUN'), ('sat','VERB'), ('on','PREP'), ('the','DET'), ('mat','NOUN')],
    [('A','DET'), ('dog','NOUN'), ('barked', 'VERB')],
]

X_train = [sentence_features(sentence) for sentence in train_data]
y_train = [sentence_labels(sentence) for sentence in train_data]

# Train CRF model
crf = sklearn_crfsuite.CRF(algorithm='lbfgs')
crf.fit(X_train, y_train)


CRF(algorithm='lbfgs')

In [ ]:
test_sentence = [('The'), ('man'),('running')]
X_test = [sentence_features(test_sentence)]
y_pred = crf.predict(X_test)
print(y_pred[0])  # Predicted POS tags


['DET' 'NOUN' 'VERB']
